In [1]:
import numpy as np 
import pandas as pd 
import os
import datetime

In [23]:
df = pd.read_xml('../input/infoce/InfoCept Participants Data/infocept_xml_data_.xml')

### 1. Process the "start_time_ts" column to create "time_spec_points" column. Allot exact points equaling the time of the day the user is surfing, but allot extra 5 points if they are surfing after 4 o clock in the afternoon. For example: If the user starts surfing at 9 o clock in the morning allot 9 points, but user surfing after 4 o clock in the afternoon will receive 21 points.

In [24]:
df['time_spec_points'] = df.start_time_ts.apply(lambda x: x[-8:])
df['time_spec_points'] = pd.to_datetime(df['time_spec_points'], format='%H:%M:%S').dt.time

In [26]:
df["time_spec_points"]

In [27]:
df["time_spec_points"] = df.time_spec_points.apply(lambda x:x.hour if x<datetime.time(16, 0, 0) else x.hour+5)

In [28]:
df["time_spec_points"] 

### 2. Reduce the "total_duration" round to 2 decimal places: example (166.711175 -> 166.71) (0.935320 -> 0.94)

In [29]:
df["total_duration"]=df["total_duration"].round(2)

### 3.  Add the values of columns  "total_clicks", "total_items", "total_cats" to create a new column: "total_inventory". If the value "total_inventory" is greater than 10, then make sure the value is increased by 100%.

In [30]:
df[["total_clicks", "total_items", "total_cats" ]].head(5)

In [31]:
df["total_inventory"]=df[["total_clicks", "total_items", "total_cats" ]].sum(axis=1)
df["total_inventory"]

In [32]:
df["total_inventory"] = df.total_inventory.apply(lambda x:x+x if x>10 else x)
df["total_inventory"]

### 4. Create a new column "give_big_discount". The value of '"give_big_discount" should be 1 if day_of_week is 0 and is_special_day is 1, otherwise 0.

In [33]:
df.loc[(df['is_special_day'] == 1) & (df['day_of_week'] == 0)][["day_of_week", "is_special_day"]].head(5)

In [34]:
df["give_big_discount"] = df.apply(lambda x:1 if  (x["day_of_week"]==0 and x["is_special_day"]==1) else 0, axis=1)


In [35]:
df.loc[(df['is_special_day'] == 1) & (df['day_of_week'] == 0)][["day_of_week", "is_special_day", "give_big_discount"]].tail(5)

### 5. Create a new column prod_views_buys_ratio which should be a ratio of prod_views_freqs and prod_views_freqs

In [36]:
df[["prod_views_freqs", "prod_buys_freqs"]].head()

In [37]:
df["prod_views_buys_ratio"] = df.apply(lambda x:x["prod_views_freqs"]/x["prod_buys_freqs"],axis=1)

In [38]:
df["prod_views_buys_ratio"]

### 6. Create a new column create loyalty_points which should be calculated based on the following conditions: if a user is spending more than 3 seconds on a Sunday -- and loyalty points score will be 10 multiplied seconds over 3. 

In [44]:
# def time_spent(start_time, end_time):

#     df = pd.date_range(start=start_time, end=end_time).to_series().dt.dayofweek
#     df = df.to_frame().reset_index(drop=False)
#     df.loc[len(df)-1,'index'] = end_time
#     first_index = df.index[0]
#     last_index = df.index[-1]
#     df = df[df[0]==6]
    

#     if first_index in df.index or last_index in df.index:

#         temp = df.iloc[1:-1,:]
#         middle = len(temp)*86400
    
#         first = datetime.timedelta(hours=df.iloc[0]['index'].hour, minutes=df.iloc[0]['index'].minute, seconds=df.iloc[0]['index'].second)-datetime.timedelta(hours=0, minutes=0, seconds=0)
#         last = datetime.timedelta(hours=df.iloc[-1]['index'].hour, minutes=df.iloc[-1]['index'].minute, seconds=df.iloc[-1]['index'].second)-datetime.timedelta(hours=0, minutes=0, seconds=0)
        
#         total = first.seconds + last.seconds+middle
#     else:

#         total = len(df)*86400
#     return  total

# df["loyalty_points"] =  df.apply(lambda x:time_spent(pd.to_datetime(x['start_time_ts']), pd.to_datetime(x['end_time_ts'])),axis=1)

In [55]:
df["loyalty_points"] = df.apply(lambda x:10*(x["total_duration"]-3) if x["total_duration"]>3 and x["day_of_week"]==0 else 0, axis=1)

In [56]:
df["loyalty_points"]

In [57]:
subm = pd.read_csv("../input/infoce/InfoCept Participants Data/infocept_submission.csv")

In [58]:
df[subm.columns.to_list()].describe()

In [59]:
df[subm.columns.to_list()]

In [60]:
df[subm.columns.to_list()].to_csv('my_submission1.csv', index=False)